In [ ]:
import io
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
data_frame = pd.read_csv('../content/train.csv')
data_frame['pickup_time'] = pd.to_datetime(data_frame['pickup_time'])
data_frame['drop_time'] = pd.to_datetime(data_frame['drop_time'])
data_frame['trip_duration'] = data_frame['drop_time'] - data_frame['pickup_time']
data_frame = data_frame.drop(['drop_time', 'pickup_time'], axis='columns')
data_frame.label = data_frame.label.map({"correct":1,"incorrect":0})

In [ ]:
import numpy as np
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputed_X = pd.DataFrame(my_imputer.fit_transform(data_frame.drop(['label', 'trip_duration'], axis='columns')))
imputed_X.columns = data_frame.drop(['label', 'trip_duration'],axis='columns').columns
imputed_X.head()

In [ ]:
from math import radians, cos, sin, asin, sqrt

def distance(lat1, lat2, lon1, lon2):
  dist_array = []
  for i in range (len(lat1)):
    # The math module contains a function named 
    # radians which converts from degrees to radians. 
    lon1_i = radians(lon1[i]) 
    lon2_i = radians(lon2[i]) 
    lat1_i = radians(lat1[i]) 
    lat2_i = radians(lat2[i])
        
    # Haversine formula  
    dlon = lon2_i - lon1_i  
    dlat = lat2_i - lat1_i 
    a = sin(dlat / 2)**2 + cos(lat1_i) * cos(lat2_i) * sin(dlon / 2)**2

    c = 2 * asin(sqrt(a))  
      
    # Radius of earth in kilometers. Use 3956 for miles 
    r = 6371
        
    # calculate the result 
    dist = c * r
    dist_array.append(dist)
  return dist_array

imputed_X['distance'] = distance(imputed_X['pick_lat'], imputed_X['drop_lat'], imputed_X['pick_lon'], imputed_X['drop_lon'])
imputed_X['trip_duration'] = data_frame['trip_duration'] / np.timedelta64(1, 's')
imputed_X['syn_1'] = (imputed_X['fare'] - imputed_X['meter_waiting_fare']) / imputed_X['distance']
imputed_X['syn_2'] = (imputed_X['fare'] - imputed_X['meter_waiting_fare']) / (imputed_X['duration'] - imputed_X['meter_waiting'])
imputed_X['syn_3'] = imputed_X['distance'] / (imputed_X['duration'] - imputed_X['meter_waiting'])
imputed_X['syn_4'] = imputed_X['meter_waiting'] / imputed_X['duration']
imputed_X['syn_5'] = imputed_X['additional_fare'] / imputed_X['distance']
imputed_X['syn_6'] = imputed_X['additional_fare'] / (imputed_X['fare'] + imputed_X['additional_fare'])
imputed_X['syn_7'] = (imputed_X['duration'] - imputed_X['meter_waiting']) / imputed_X['fare']
imputed_X.head()

In [ ]:
# import matplotlib.pyplot as plt

# fig, ax = plt.subplots(figsize=(16,8))
# ax.scatter(data_frame['meter_waiting_till_pickup'], data_frame['fare'])
# ax.set_xlabel('meter_waiting_till_pickup')
# ax.set_ylabel('Fare')
# plt.show()
imputed_X = imputed_X.replace([np.inf, -np.inf], np.nan)
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputed_X_new = pd.DataFrame(my_imputer.fit_transform(imputed_X))
imputed_X_new.columns = imputed_X.columns
imputed_X_new.head()

In [ ]:
from sklearn.model_selection import train_test_split

X = imputed_X_new.drop(['tripid', 'pick_lat', 'pick_lon',	'drop_lat',	'drop_lon'], axis='columns')
y = data_frame['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
# from imblearn.over_sampling import RandomOverSampler

# ros = RandomOverSampler(random_state=0)
# X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
# # X_resampled.columns = X_train.columns
# # y_resampled.columns = y_train.columns

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
# a=[0, 0.001, 0.005, 0.01, 0.05]
for i in range(1, 100, 1):
  j = i/100
  # classifier = RandomForestClassifier(max_depth=i, random_state=0)
  # classifier = KNeighborsClassifier(n_neighbors=i)
  classifier = XGBClassifier(learning_rate = j)
  classifier.fit(X_train, y_train)
  # X_test.columns = ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14']
  preds = classifier.predict(X_test)
  print("Depth - {} , score {}".format(i,f1_score(y_test, preds, average='macro')))

In [ ]:
# classifier = RandomForestClassifier(max_depth=18, random_state=0)
# classifier = KNeighborsClassifier(n_neighbors=4)
classifier = XGBClassifier(learning_rate = 0.74)
classifier.fit(X_train, y_train)

In [ ]:
# uploaded = files.upload()
test_data = pd.read_csv(io.BytesIO(uploaded['test.csv']))

In [ ]:
test_data['pickup_time'] = pd.to_datetime(test_data['pickup_time'])
test_data['drop_time'] = pd.to_datetime(test_data['drop_time'])
X = test_data.drop(['drop_time', 'pickup_time', 'tripid', 'pick_lat', 'pick_lon',	'drop_lat',	'drop_lon'], axis='columns')
X['distance'] = distance(test_data['pick_lat'], test_data['drop_lat'], test_data['pick_lon'], test_data['drop_lon'])
test_data['trip_duration'] = test_data['drop_time'] - test_data['pickup_time']
X['trip_duration'] = test_data['trip_duration'] / np.timedelta64(1, 's')

X['syn_1'] = (X['fare'] - X['meter_waiting_fare']) / X['distance']
X['syn_2'] = (X['fare'] - X['meter_waiting_fare']) / (X['duration'] - X['meter_waiting'])
X['syn_3'] = X['distance'] / (X['duration'] - X['meter_waiting'])
X['syn_4'] = X['meter_waiting'] / X['duration']
X['syn_5'] = X['additional_fare'] / X['distance']
X['syn_6'] = X['additional_fare'] / (X['fare'] + X['additional_fare'])
X['syn_7'] = (X['duration'] - X['meter_waiting']) / X['fare']
# X.columns = ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14']
X.head()

In [ ]:
predictions = classifier.predict(X)

In [ ]:
results = pd.DataFrame(predictions)
results.columns = ['prediction']
results['tripid'] = test_data['tripid']
f_cols = ['tripid','prediction']
results = results[f_cols]
results.to_csv('../content/submission10.csv',index=False)